<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Content-based algorithm</h1> 
<h2 align="center">Alejo Paullier</h2> 

In [1]:
%%time
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])

ratings = ratings.drop(['timestamp'],axis=1)

ratings.head(5)

# Boolean ratings

boolean = ratings.copy()
boolean['rating'] = boolean['rating'].replace([1,2,3,4,5],1)

# Genres Matrix

m_cols = ['movie id','movie title','release date','video release date','IMDb URL',
          'unknown','Action','Adventure','Animation','Children','Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
          'Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 100k\\u.item', sep='|', names=m_cols,encoding='latin-1')
movies = movies.set_index('movie id')
movies = movies.drop(['video release date'],axis = 1)
genres = movies.drop(['movie title','release date','IMDb URL'],axis = 1)
genres = genres.divide(((((genres.T)**2).sum())**(1/2)),axis=0)

# Unpivot Ratings Matrix

ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
ratings_copy = ratings.copy()
OldMin = 1
OldMax = 5
NewMin = -1
NewMax = 1
ratings = (((ratings - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
ratings = ratings.fillna(0)
ratings

# Unpivot Boolean matrix

boolean = boolean.pivot(index = 'user',columns='item',values ='rating')
boolean = boolean.fillna(0)


# Profile matrix

np.seterr(divide='ignore', invalid='ignore')
dot = boolean.values.dot(genres.values)
dot = pd.DataFrame(dot)
profile = pd.DataFrame((ratings).values.dot(genres.values), columns=genres.columns, index=(ratings).index)
profile = profile.values/(dot.values)
profile = pd.DataFrame(profile)
profile.columns = genres.columns
profile = profile.fillna(0)


# Compute Predictions (Genres . Profile^T)

count = genres.sum(axis=1)
count = (pd.DataFrame(count))
prediction = pd.DataFrame((genres.values.dot((profile.T).values))/count.values, columns=(profile.T).columns, index=genres.index)
prediction = prediction.T
prediction.columns = ratings.columns
prediction.index = ratings.index

# Reset to 1-5 star scale

OldMin = -1
OldMax = 1
NewMin = 1
NewMax = 5
prediction = (((prediction - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin


Wall time: 8.59 s


In [2]:
%%time
MAE = abs(ratings_copy-prediction)
MAE.mean(axis=1).mean()

Wall time: 162 ms


0.7274366675243344

In [3]:
%%time
RMSE = ((((MAE**2).mean(axis=1)))**(1/2)).mean()
RMSE

Wall time: 360 ms


0.9022961851507607